In [55]:
from ultralytics import YOLO
import cv2
import torch
from collections import defaultdict

In [56]:
path_to_weights = '../sandboxes/best.pt'
model = YOLO(path_to_weights)
device = 'mps:0' if torch.backends.mps.is_available() else 'cuda:0' if torch.cuda.is_available() else 'cpu'

video ='https://hd-auth.skylinewebcams.com/live.m3u8?a=67sg4qmmrche0a4hrkmkr8g6j2'

HEIGHT = 640
WIDTH = 480


cap = cv2.VideoCapture(video)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)


cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)

print('Video Info' + '  FPS: '+str(int(fps))+' Width: '+str(HEIGHT)+' Height: '+str(WIDTH))



# Store the track history
track_history = defaultdict(lambda: [])

while cap.isOpened():
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True,classes=[0],conf=0.5,device=device,verbose=False)
        for result in results:
            if result.boxes is None or result.boxes.id is None:
                continue
            # Get the boxes and track IDs
            else:
                boxes = result.boxes.xywh.cpu()
                track_ids = result.boxes.id.cpu().numpy().astype(int)
                # Plot the tracks and draw bounding boxes without class annotations
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    # Calculate top left corner of the bounding box
                    top_left = int(x - w / 2), int(y - h / 2)
                    bottom_right = int(x + w / 2), int(y + h / 2)
                    
                    # Draw bounding box
                    cv2.rectangle(frame, top_left, bottom_right, (0, 0, 255), 2)
                    
                    track = track_history[track_id]
                    track.append((int(x), int(y)))  # x, y center point
                    if len(track) > 30:
                        track.pop(0)
                    
                    # Draw the tracking lines
                    if len(track) > 1:
                        for i in range(1, len(track)):
                            cv2.line(frame, track[i - 1], track[i], (0, 255, 0), 2)
                # Display the annotated frame
                cv2.imshow("YOLOv8 Tracking", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break
cap.release()
cv2.destroyAllWindows()        

Video Info  FPS: 12 Width: 640 Height: 480


In [58]:
# Assuming the YOLO and model initialization code is defined elsewhere

path_to_weights = '../sandboxes/crosswalk_detect.pt'
model = YOLO(path_to_weights)
device = 'mps:0' if torch.backends.mps.is_available() else 'cuda:0' if torch.cuda.is_available() else 'cpu'

video ='https://hd-auth.skylinewebcams.com/live.m3u8?a=67sg4qmmrche0a4hrkmkr8g6j2'

HEIGHT = 640
WIDTH = 480

cap = cv2.VideoCapture(video)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)

print(f'Video Info - FPS: {int(fps)} Width: {HEIGHT} Height: {WIDTH}')

# Flag to check if inference was already done
inference_done = False
results = None  # To store the inference results

while cap.isOpened():
    success, frame = cap.read()

    if success:
        if not inference_done:
            # Run YOLOv8 inference on the first frame
            results = model(frame, conf=0.2, device=device, verbose=False)
            inference_done = True
        # Visualize the results on every frame
        if results is not None:
            for result in results[0].boxes:  # Loop through detections
                # Extract bounding box coordinates
                xmin, ymin, xmax, ymax = result.xyxy.cpu().numpy()[0].astype(int)
                conf = result.conf.cpu().numpy().astype(float)
                cls = result.cls.cpu().numpy().astype(int)
                label = model.names[int(cls)]  

                # Draw bounding box and label on the frame
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                cv2.putText(frame, label, (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

        # Display the frame with detections
        cv2.imshow("YOLOv8 Inference", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

Video Info - FPS: 12 Width: 640 Height: 480


/var/folders/vl/_q2t9dnn149gm7fk6whh5f080000gn/T/ipykernel_39130/2387429360.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = model.names[int(cls)]
